In [1]:
!pip install tensorflow keras matplotlib

In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [18]:
# Define constants
IMAGE_SIZE = (640, 640)
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCHS = 50

In [19]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [20]:
# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Tea',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 345 images belonging to 4 classes.


In [21]:
val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Tea',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 84 images belonging to 4 classes.


In [22]:
# Load pre-trained MobileNetV2 model without the top classification layer
base_model = MobileNetV2(input_shape=(640, 640, 3), weights='imagenet', include_top=False)

In [23]:
# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [24]:
# Combine base model with custom head
model = Model(inputs=base_model.input, outputs=predictions)

In [25]:
# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

In [26]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    verbose=1
)

Epoch 1/50
11/11 [==============================] - 236s 21s/step - loss: 1.2479 - accuracy: 0.5420 - val_loss: 0.7838 - val_accuracy: 0.5714
Epoch 2/50
11/11 [==============================] - 215s 19s/step - loss: 0.5977 - accuracy: 0.7391 - val_loss: 0.7437 - val_accuracy: 0.7262
Epoch 3/50
11/11 [==============================] - 251s 23s/step - loss: 0.5176 - accuracy: 0.7449 - val_loss: 0.4564 - val_accuracy: 0.7857
Epoch 4/50
11/11 [==============================] - 252s 23s/step - loss: 0.3305 - accuracy: 0.8696 - val_loss: 0.4190 - val_accuracy: 0.8690
Epoch 5/50
11/11 [==============================] - 212s 19s/step - loss: 0.2724 - accuracy: 0.9014 - val_loss: 0.3922 - val_accuracy: 0.8333
Epoch 6/50
11/11 [==============================] - 209s 19s/step - loss: 0.2948 - accuracy: 0.8725 - val_loss: 0.4013 - val_accuracy: 0.8214
Epoch 7/50
11/11 [==============================] - 209s 19s/step - loss: 0.2254 - accuracy: 0.9101 - val_loss: 0.2884 - val_accuracy: 0.8929
Epoch 

In [29]:
# prompt: what is the accuracy of the model

accuracy = history.history['accuracy'][-1]
print('Accuracy:', accuracy * 100, "%")


Accuracy: 94.49275135993958 %


In [30]:
# Save the trained model
model.save('tea_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
